# Step Function

In [10]:
import numpy as np

def step_function(x) :
    if x > 0 :
        return 1
    else :
        return 0

print(step_function(0))
print(step_function(2))
print(step_function(3.14))
# print(step_function(np.array([1.0, 2.0, 3.3])))

0
1
1


In [11]:
import numpy as np

def step_function(x):
    x = np.array(x)  # Ensure that x is a numpy array
    y = x > 0
    return y.astype(np.int32)

print(step_function(0))
print(step_function(2))
print(step_function(3.14))
print(step_function(np.array([1.0, 2.0, 3.3])))


0
1
1
[1 1 1]


# Sigmoid Function 

In [12]:
import numpy as np

def sigmoid(x) :
    return 1 / (1 + np.exp(-x))

x = np.array([-100.0, -1.0, 0.0, 1.0, 200.0])
sigmoid(x)

array([3.72007598e-44, 2.68941421e-01, 5.00000000e-01, 7.31058579e-01,
       1.00000000e+00])

# ReLU (Rectified Linear Unit)

In [13]:
def relu(x) :
    return np.maximum(0, x)

print(relu(-1))
print(relu(0))
print(relu(0.1))
print(relu(100))

0
0
0.1
100


# 두 행렬의 곱 : np.dot()

In [14]:
A = np.array([
    [1, 2], [3, 4], [5, 6]
])
B = np.array([7, 8])

print(A.shape)
print(B.shape)

print("A * B = "+ str(np.dot(A, B)))

(3, 2)
(2,)
A * B = [23 53 83]


# 각 층의 신호 전달 구현하기

In [15]:
import numpy as np

def sigmoid_function(x) :
    return 1 / (1 + np.exp(-x))
def identity_function(x) :
    return x

# 신경망 구현의 관례에 따라 가중치만 대문자(W)로 쓰고, 그 외 편향과 중간 결과 등은 모두 소문자로 작성.
def init_network() :
    network = {}
    network['W1'] = np.array([ [0.1, 0.3, 0.5], [0.2, 0.4, 0.6] ])
    network['b1'] = np.array([ 0.1, 0.2, 0.3 ])
    network['W2'] = np.array([ [0.1, 0.4], [0.2, 0.5], [0.3, 0.6] ])
    network['b2'] = np.array([ 0.1, 0.2 ])
    network['W3'] = np.array([ [0.1, 0.3], [0.2, 0.4] ])
    network['b3'] = np.array([ 0.1, 0.2 ])
    return network

# 신호가 순방향(입력에서 출력 방향)으로 전달됨(순전파)을 알리기 위해 forward()
# 신경망 학습을 다룰 때, 역방향(backward, 출력에서 입력 방향)처리도 있다.
def forward(network, x) :
    W1, W2, W3 = network['W1'], network['W2'], network['W3']
    b1, b2, b3 = network['b1'], network['b2'], network['b3']

    # Layer 0 (Input Layer)
    a1 = np.dot(x, W1) + b1
    z1 = sigmoid_function(a1)
    # Layer 1 (Hidden Layer)
    a2 = np.dot(z1, W2) + b2
    z2 = sigmoid_function(a2)
    # Layer 2 (Output Layer)
    a3 = np.dot(z2, W3) + b3
    y = identity_function(a3)
    return y

network = init_network()
x = np.array( [1.0, 0.5] )
y = forward(network, x)
print(y)

[0.31682708 0.69627909]


# Softmax Function

In [16]:
import numpy as np 

def softmax_function(a) :
    c = np.max(a)  # 입력 신호 중 최대값을 c
    exp_a = np.exp(a - c) # Overflow를 막기 위해 모든 입력 신호에서 최대값을 빼줌
    sum_exp_a = np.sum(exp_a)
    y = exp_a / sum_exp_a
    return y

a = np.array([ 0.3, 2.9, 4.0 ])
y = softmax_function(a)
print(y)
print(np.sum(y))

[0.01821127 0.24519181 0.73659691]
1.0


# MNIST Dataset

## Data Load

In [25]:
import sys, os
from dataset.mnist import load_mnist

(x_train, y_train), (x_test, y_test) = load_mnist(normalize=True, flatten=True, one_hot_label=False)

print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(x_test.shape)

(60000, 784)
(60000,)
(10000, 784)
(10000, 784)


In [27]:
import sys, os 
import numpy as np
from dataset.mnist import load_mnist
from PIL import Image
import pickle

def img_show(img) :
    pil_img = Image.fromarray(np.uint8(img))
    pil_img.show()
    
(x_train, y_train), (x_test, y_test) = load_mnist(normalize=True, flatten=True)

img = x_train[0]
label = y_train[0]
print(label)

print(img.shape)
img = img.reshape(28, 28)
print(img.shape)

img_show(img)


5
(784,)
(28, 28)


## 신경망의 추론 처리

* 입력층 뉴런을 784개 (28 x 28)
* 출력층 뉴런을 10개
* 은닉층 2개(첫 번째 은닉층 : 50개 뉴런, 두 번째 은닉층 : 100개 뉴런)

In [32]:
import sys, os 
import numpy as np
from dataset.mnist import load_mnist
from PIL import Image
import pickle

# 데이터 불러오기
def get_data() :
    (x_train, y_train), (x_test, y_test) = load_mnist(normalize=True, flatten=True, one_hot_label=False) # 정규화(전처리)
    return x_test, y_test 

# 신경망 구축
def init_network() :
    with open("./deep-learning-from-scratch/ch03/sample_weight.pkl", 'rb') as f :
        network = pickle.load(f) # '미리 학습된 가중치 매개변수'를 읽는다
    return network

# 출력층 결과 받기
def predict(network, x) :
    # 미리 학습된 가중치(신경망, 편향)들을 각각 행렬화, 벡터화
    W1, W2, W3 = network['W1'], network['W2'], network['W3']
    b1, b2, b3 = network['b1'], network['b2'], network['b3']
    
    # 각 층과 활성화 함수 거치기
    a1 = np.dot(x, W1) + b1
    z1 = sigmoid(a1)
    a2 = np.dot(z1, W2) + b2
    z2 = sigmoid(a2)
    a3 = np.dot(z2, W3) + b3
    z3 = sigmoid(a3)
    y = softmax_function(a3)
    return y

x, t = get_data()
network = init_network()

accuracy_cnt = 0
for i in range(len(x)) :
    y = predict(network, x[i])
    p = np.argmax(y) # 확률이 가장 높은 원소의 인덱스를 얻는다
    if p == t[i] : 
        accuracy_cnt += 1
    
print("Accuracy : " + str(float(accuracy_cnt) / len(x)))
    

Accuracy : 0.9352


## Batch 처리하기

In [34]:
x, _ = get_data()
network = init_network()

W1, W2, W3 = network['W1'], network['W2'], network['W3']

print(x.shape)  # (10000, 784)
print(W1.shape) #        (784, 50)
print(W2.shape) #             (50, 100)
print(W3.shape) #                 (100, 10)

# 입력 데이터의 shape : 100 * 784, 출력 데이터의 shape : 100 * 10
# 이는 100장 분량 입력 데이터의 결과가 한 번에 출력됨을 나타낸다. --> 이처럼 하나로 묶은 입력 데이터를 '배치(batch)' 라고 한다.

(10000, 784)
(784, 50)
(50, 100)
(100, 10)


In [43]:
x, y = get_data() 
network = init_network()

accuracy_cnt = 0

# Batch 처리한 부분 ---------------------------------------------------------
batch_size = 100
for i in range(0, len(x), batch_size) :
    x_batch = x[i : i+batch_size] # 입력 데이터를 100개씩 묶는다
    y_batch = predict(network, x_batch)
    p = np.argmax(y_batch, axis=1) # 최대값의 인덱스 반환
    accuracy_cnt += np.sum(p == y[i : i+batch_size]) 
# ------------------------------------------------------------------------
    
print("Accuracy : " + str(float(accuracy_cnt) / len(x)))

Accuracy : 0.9352
